# Assignment 1: Transformations


TEAM-ID:   
TEAM-NAME:   
YOUR-ID:   
YOUR-NAME:    

(Although you work in groups, both the students have to submit to Moodle, hence there's name field above)

# Instructions
- Please check Moodle for "TEAM-ID" and "TEAM-NAME" fields above. Some of your names have been edited because of redundancy/simplicity. Instructions for submitting the assignment through GitHub Classrooms/Moodle has been uploaded on Moodle. Any clarifications will be made there itself.
- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. See `./misc/installation` for detailed step-by-step instructions about the installation setup.
- Both the team members must submit the zip file.
- For this assignment, you will be using Open3D extensively. Refer to [Open3D Documentation](http://www.open3d.org/docs/release/): you can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. Make sure your code is modular since you will be reusing them for future assignments.
- Take a look at the entire assignment. The descriptive questions at the end might have a clue if you are stuck somewhere.
- Answer the descriptive questions in your own words with context & clarity. Do not just copy-paste from some Wikipedia page. You will be evaluated accordingly.
- Please call the visualization functions only when they are asked. They are asked explicitly at the end of every section.
- You could split the Jupyter Notebook cells where `TODO` is written, but please try to avoid splitting/changing the structure of other cells.

In [1]:
import open3d as o3d
import time
import copy
import numpy as np
from scipy.linalg import logm
from scipy.spatial.transform import Rotation as R
from scipy.optimize import fsolve

# 1. Getting started with Open3D
## 1.1 Converting RGBD image into Point Cloud


In your robotics journey, it is common to be given just the depth images along with camera parameters in a generic dataset and you'd want to build a 3D data representation out of it, for example, a point cloud. You will understand the math behind these concepts in detail during Vision classes, for now, you can use the in-built functions as a black box.
- Below are the given RGB and D images from SUN RGB-D dataset ([S.Song, CVPR 2015](https://rgbd.cs.princeton.edu/)). 
![low-res-RGBD.resized.png](./misc/low-res-RGBD.png)

- Read these two images `color.jpg` and `depth.png` given in current folder using Open3D. Convert it to a point cloud using the default camera parameters (`o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault`).
Then,
- Create a "world" frame $A$ and combine this (just use $+$ operator) with the above point cloud and save it as `scene.pcd`. Put it aside for now.
- Write a simple function `one_one` to visualize `scene.pcd`.

In [2]:
##############################################################################
# TODO: Do tasks described in 1.1                                            #
##############################################################################

# CONSTANTS
NUM_SAMPLES = 2000
INIT_FACTOR = 5

color_scene = o3d.io.read_image('color.jpg')
depth_scene = o3d.io.read_image('depth.png')

pinhole_camera_intrinsic = o3d.camera.PinholeCameraIntrinsic(o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)

rgbd_scene = o3d.geometry.RGBDImage.create_from_sun_format(color_scene, depth_scene, convert_rgb_to_intensity=False)
pcd_scene = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_scene, pinhole_camera_intrinsic)
Rot = pcd_scene.get_rotation_matrix_from_zyx((0, 0, np.pi))
pcd_scene.rotate(Rot, center=(0, 0, 0))


#World Frame
frame_world = o3d.geometry.TriangleMesh.create_coordinate_frame()
frame_world = frame_world.sample_points_poisson_disk(number_of_points=NUM_SAMPLES, init_factor=INIT_FACTOR)
pcd_scene = pcd_scene + frame_world

o3d.io.write_point_cloud('scene.pcd', pcd_scene)

def one_one(pcd):
    o3d.visualization.draw_geometries([pcd])
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

### Question for 1.1

- In the next code cell, call the function `one_one` here showing `scene.pcd`.

In [3]:
#uncomment the following and add input parameters if any
pcd_scene = o3d.io.read_point_cloud('scene.pcd')
one_one(pcd_scene)

# 2. Rotations, Euler angles and Gimbal Lock

## 2.1 Rotating an object

The objective here is to roughly simulate an object moving on a ground.

- Generate a cube at some point on the ground and create another frame $B$ at the center of this object. Combine these both as a single point cloud `cube.pcd`. (You can pick a point on the ground by using the `get_picked_points` method of the class `open3d.visualization.VisualizerWithEditing`.)
- Now read both the point clouds `scene.pcd` and `cube.pcd` in a script. Whatever tasks you do below are on the object `cube.pcd` (along with the axes $B$) with `scene.pcd` in the background (static).
- Given a sequence of **ZYX Euler** angles $[30^{\circ}, 90^{\circ}, 45^{\circ}]$, generate the rotation. In our case, our object (with its respective axis) undergoes rotation with the background being fixed (with its respective axis).
- Note: Throughout this assignment, we will be using the standard **ZYX** Euler angle convention.
- Write a function `two_one` to show the above by **animation** (cube rotating along each axis one by one).
    - *Hint: Use Open3D's non-blocking visualization and discretize the rotation to simulate the animation. For example, if you want to rotate by $30^{\circ}$ around a particular axis, do in increments of $5^{\circ}$ 6 times to make it look like an animation.*

In [4]:
##############################################################################
# TODO: Do tasks described in 2.1                                            #
##############################################################################

# CONSTANTS
CUBE_COLOR = [194/255, 122/255, 192/255]
CUBE_SIZE = 0.4
CUBE_CENTER = [-CUBE_SIZE/2 for i in range(3)]
FRAME_SIZE = 0.6
NUM_SAMPLES = 5000

# Create the cube
mesh_cube = o3d.geometry.TriangleMesh.create_box(width=CUBE_SIZE, height=CUBE_SIZE, depth=CUBE_SIZE)
mesh_cube.paint_uniform_color(CUBE_COLOR)

mesh_frame_cube = o3d.geometry.TriangleMesh.create_coordinate_frame(size=FRAME_SIZE, origin=CUBE_CENTER)
framed_cube = (mesh_cube + mesh_frame_cube).sample_points_poisson_disk(number_of_points = NUM_SAMPLES, init_factor=INIT_FACTOR)

o3d.io.write_point_cloud('cube.pcd', framed_cube)

scene = o3d.io.read_point_cloud('scene.pcd')
cube = o3d.io.read_point_cloud('cube.pcd')

In [5]:
def pick_points(pcd):
    print('Pick points using SHIFT + Left Click. After the point is picked a circle will be formed around that point. Press q to exit the window.')
    print('Press ENTER to begin.')
    input()
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window()
    vis.add_geometry(pcd)
    vis.run()
    vis.destroy_window()
    points = vis.get_picked_points()
    if len(points) == 0:
        raise ValueError('No points chosen.')
    return [pcd.points[i] for i in points]

In [6]:
cube_corner = pick_points(scene)[0]
cube.translate(cube_corner)
o3d.visualization.draw_geometries([scene, cube])

##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

Pick points using SHIFT + Left Click. After the point is picked a circle will be formed around that point. Press q to exit the window.
Press ENTER to begin.

[Open3D INFO] Picked point #189171 (-0.28, -0.93, -3.3) to add in queue.


In [7]:
def two_one(scene_path,cube_path):
    scene = o3d.io.read_point_cloud(scene_path)
    cube = o3d.io.read_point_cloud(cube_path)
    cube.translate(cube_corner)
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(scene)
    vis.add_geometry(cube)
    val = (np.pi)/36
    
    ROA = cube.get_rotation_matrix_from_zyx((val,0,0))
    for i in range(0,6):
        cube.rotate(ROA, center = CUBE_CENTER)
        vis.update_geometry(cube)
        vis.poll_events()
        vis.update_renderer()
        time.sleep(0.1)
    
    ROA = cube.get_rotation_matrix_from_zyx((val*6,0,0))
    ROB = cube.get_rotation_matrix_from_zyx((val*6,val,0))
    ROAT = ROA.transpose()
    RAB = np.matmul(ROB, ROAT)
    for i in range(0,18):
        cube.rotate(RAB, center = CUBE_CENTER)
        vis.update_geometry(cube)
        vis.poll_events()
        vis.update_renderer()
        time.sleep(0.1)
    
    ROB = cube.get_rotation_matrix_from_zyx((val*6, val*18, 0))
    ROC = cube.get_rotation_matrix_from_zyx((val*6, val*18, val))
    ROBT = ROB.transpose()
    RBC = np.matmul(ROC, ROBT)
    for i in range(0,9):
        cube.rotate(RBC, center = CUBE_CENTER)
        vis.update_geometry(cube)
        vis.poll_events()
        vis.update_renderer()
        time.sleep(0.1)
        
    vis.destroy_window()

### Question for 2.1

- In the next code cell, call the function `two_one` here showing the animation described in section 2.1.

In [8]:
#uncomment the following and add input parameters if any
two_one('scene.pcd','cube.pcd')

## 2.2 Euler angle & Gimbal lock

Code the following yourself from scratch (Refer Craig book - Section: $Z-Y-X$ Euler angles - same conventions/notations followed).

- Case 1: Given the rotation matrix $M_{given}$ below, extract Euler angles $\alpha , \beta ,\gamma$. Convert it back to the rotation matrix $M_{recovered}$ from Euler angles.

    $$M(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0.26200263 & -0.19674724 & 0.944799 \\0.21984631 & 0.96542533 & 0.14007684 \\
    -0.93969262 & 0.17101007 & 0.29619813\end{array}\right] $$

    After coding it from scratch, check your calculations using `scipy.spatial.transform.Rotation`. (Mandatory)

- Case 2: Given the rotation matrix $N_{given}$, extract Euler angles, and convert back $N_{recovered}$.

    $$N(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0 & -0.173648178 &  0.984807753 \\0 & 0.984807753 & 0.173648178 \\
    -1 & 0 & 0\end{array}\right] $$

    Again use `scipy` and check its output. If `scipy` is showing any warnings on any of the above cases, explain it in "**Questions for 2.2**" (last question). Write code in the next cell.
    
- (Optional) Case 3: Do the above two for quaternion using scipy functions, i.e. given the rotation matrix, extract quaternion and convert back.

In [9]:
##############################################################################
# DON'T EDIT
M_given =  np.array([[0.26200263, -0.19674724, 0.944799],
                     [0.21984631, 0.96542533, 0.14007684],
                     [-0.93969262, 0.17101007, 0.29619813]])

N_given = np.array([[0,-0.173648178,0.984807753],
                    [0, 0.984807753, 0.173648178],
                    [-1, 0, 0]])

In [10]:
# TODO: Do tasks described in 2.2                                            #
##############################################################################

def euler_zyx_from_rotation(R):
    beta = np.arctan2(-R[2, 0], np.sqrt(R[0, 0] ** 2 + R[1, 0] ** 2))
    alpha = np.arctan2(R[1, 0]/np.cos(beta), R[0, 0]/np.cos(beta))
    gamma = np.arctan2(R[2, 1]/np.cos(beta), R[2, 2]/np.cos(beta))
    return (alpha, beta, gamma)

def rotation_from_euler_zyx(alpha, beta, gamma):
    R = np.zeros((3, 3), dtype='double')
    
    R[0, 0] = np.cos(alpha) * np.cos(beta)
    R[0, 1] = np.cos(alpha) * np.sin(beta) * np.sin(gamma) - np.sin(alpha) * np.cos(gamma)
    R[0, 2] = np.cos(alpha) * np.sin(beta) * np.cos(gamma) + np.sin(alpha) * np.sin(gamma)
    
    R[1, 0] = np.sin(alpha) * np.cos(beta)
    R[1, 1] = np.sin(alpha) * np.sin(beta) * np.sin(gamma) + np.cos(alpha) * np.cos(gamma)
    R[1, 2] = np.sin(alpha) * np.sin(beta) * np.cos(gamma) - np.cos(alpha) * np.sin(gamma)
    
    R[2, 0] = -np.sin(beta)
    R[2, 1] =  np.cos(beta) * np.sin(gamma)
    R[2, 2] =  np.cos(beta) * np.cos(gamma)
    
    return R

In [11]:
# Case 1

# Ours
print('Ours:')
euler_M = euler_zyx_from_rotation(M_given)
print('Euler angles (zyx): ', np.array(euler_M))
M_generated = rotation_from_euler_zyx(*euler_M)
print('Recovered rotation matrix:\n', M_generated)

# Scipy's
print('\nScipy\'s:')
bench = R.from_matrix(M_given)
print('Euler angles (zyx): ', bench.as_euler('ZYX'))
bench2 = R.from_euler('ZYX', bench.as_euler('ZYX'))
print('Recovered rotation matrix:\n', bench2.as_matrix())

Ours:
Euler angles (zyx):  [0.6981317  1.22173048 0.52359878]
Recovered rotation matrix:
 [[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542534  0.14007684]
 [-0.93969262  0.17101007  0.29619813]]

Scipy's:
Euler angles (zyx):  [0.69813171 1.22173048 0.52359878]
Recovered rotation matrix:
 [[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542534  0.14007684]
 [-0.93969262  0.17101007  0.29619813]]


In [12]:
# Case 2

# Ours
print('Ours:')
euler_N = euler_zyx_from_rotation(N_given)
print('Euler angles (zyx): ', np.array(euler_N))
N_generated = rotation_from_euler_zyx(*euler_N)
print('Recovered rotation matrix:\n', N_generated)

# Scipy's
print('\nScipy\'s:')
bench = R.from_matrix(N_given)
print('Euler angles (zyx): ', bench.as_euler('ZYX'))
bench2 = R.from_euler('ZYX', bench.as_euler('ZYX'))
print('Recovered rotation matrix:\n', bench2.as_matrix())

Ours:
Euler angles (zyx):  [0.         1.57079633 0.        ]
Recovered rotation matrix:
 [[ 6.123234e-17  0.000000e+00  1.000000e+00]
 [ 0.000000e+00  1.000000e+00  0.000000e+00]
 [-1.000000e+00  0.000000e+00  6.123234e-17]]

Scipy's:
Euler angles (zyx):  [0.17453293 1.57079633 0.        ]
Recovered rotation matrix:
 [[ 1.66533454e-16 -1.73648178e-01  9.84807753e-01]
 [ 2.77555756e-17  9.84807753e-01  1.73648178e-01]
 [-1.00000000e+00  0.00000000e+00  1.66533454e-16]]


/home/jivitesh/anaconda3/envs/mr_assignment1/lib/python3.7/site-packages/scipy/spatial/transform/rotation.py:133: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  warnings.warn("Gimbal lock detected. Setting third angle to zero since"


In [13]:
# Case 3

# Scipy's for M
print('Scipy\'s for M:')
bench = R.from_matrix(M_given)
print('Quarternions: ', str(bench.as_quat()))
bench2 = R.from_quat(bench.as_quat())
print('Recovered rotation matrix:\n', bench2.as_matrix())

# Scipy's for N
print('\nScipy\'s for N:')
bench = R.from_matrix(N_given)
print('Quarternions: ', str(bench.as_quat()))
bench2 = R.from_quat(bench.as_quat())
print('Recovered rotation matrix:\n', bench2.as_matrix())

##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

Scipy's for M:
Quarternions:  [0.00973605 0.59313249 0.13112033 0.79429624]
Recovered rotation matrix:
 [[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542534  0.14007684]
 [-0.93969262  0.17101007  0.29619813]]

Scipy's for N:
Quarternions:  [-0.06162842  0.70441603  0.06162842  0.70441603]
Recovered rotation matrix:
 [[ 0.         -0.17364818  0.98480775]
 [ 0.          0.98480775  0.17364818]
 [-1.          0.          0.        ]]


## Questions for 2.2
- Have you used `np.arctan` or an any equivalent `atan` function above? Why or why not?   
    * Ans: We have used `np.arctan2` to calculate the inverse tangents of the given values. As opposed to a single-argument inverse tangent function, `np.arctan2` takes in $x$ and $y$ values separately, and uses them to infer the quadrant of the angle, thus being able to output angles in the complete range $[-\pi, \pi)$. Single-argument inverse tangents functions lack information about the individual signs of $x$ and $y$ and hence can only return an angle within the *principal-value range* of $[-\frac{\pi}{2}, \frac{\pi}{2}).$ Because this would have restricted the output euler angles, this was avoided.

### For Case 1 above,
- What Euler angles  $\alpha , \beta ,\gamma$ did you get? Replace `my_array_case1` with your array.

In [14]:
# Uncomment and replace my_array_case1 with your array.
print("My Euler angles for case 1 are " + str(euler_M))

My Euler angles for case 1 are (0.6981317003487097, 1.2217304765297095, 0.5235987753730347)


  - Were you able to recover back your rotation matrix when you converted it from Euler angles? Why/why not? Replace `M_given` and `M_recovered` with your matrices below and explain "why/why not" after this code snippet.

In [15]:
# Uncomment and Replace M_given and M_recovered with your matrices below.
error = np.linalg.norm(logm(M_given @ M_generated.T))
print("For case 1, it is " + str(error<0.0001) + " I could recover the original matrix.")

For case 1, it is True I could recover the original matrix.


- Why/why not? Based on your observations here, is there any problem with Euler angle representation for Case 1? If yes, what is it?

    - Ans: The problem with Euler Angles is their inability to be directly used in computations and the loss of precision that occurs while performing the mathematical conversions. While Euler angles are more compact and also easier to visualise than rotation matrices, they cannot be used to compute frame or point rotations directly, making them not-so-useful for a computer.

### Repeat the above for Case 2.

In [16]:
# Uncomment and Replace N_given and N_recovered with your matrices below.
print("My Euler angles for case 2 are " + str(euler_N))
error = np.linalg.norm(logm(N_given @ N_generated.T))
print("For case 2, it is " + str(error<0.0001) + " I could recover the original matrix.")

My Euler angles for case 2 are (0.0, 1.5707963267948966, 0.0)
For case 2, it is False I could recover the original matrix.


* Why/why not? Based on your observations here, is there any problem with Euler angle representation for Case 2? If yes, what is it?

    * We were not able to recover the rotation matrix from the Euler angles because of a gimbal lock. In case of a gimbal lock, the sequence of Euler rotations that lead to that position are not unique, hence it is not possible to uniquely determine the Euler angles. (In all other cases, Euler angles can be uniquely determined if kept within their ranges: $\alpha, \gamma \in [-\pi, \pi)$ and $\beta \in [0, \pi]$).
    * **The reason that the function failed to find even a single sequence of Euler angle rotations out of those multiple possibilities is that a gimbal lock occurs when $\beta = \frac{\pi}{2}$. In this case, $tan(\beta)\to\infty$ and** `np.arctan2` **fails to perform when provided a finite** `y` **and a vanishing** `x`**. An incorrect rotation matrix followed an incorrect set of Euler angles.**
    
    
* Explain any more problems with Euler angle representation. Explain what you understand by Gimbal lock (concisely in your own words). You could revisit this question in the section 2.4.
    * The major problem with Euler angles is the phenomenon of Gimbal lock. Its cause and consequences are the following.
    * **Cause:**\
    Because Euler angles are rotations about moving axes (i.e., while rotating a frame through a sequence of 3 rotations, each subsequent rotation is performed about the axes which are a result of the previous rotation), rotations about *outer* (or higher in the hierarchy) axes affect the *inner* (lower in the hierarchy) axes. This means that a rotation about the $y$ axis affects the $x$ axis, but not the $z$, allowing such a situation when $\beta = \frac{\pi}{2}$:
    <img src="https://user-images.githubusercontent.com/43912285/91614816-dfadc280-e99f-11ea-8612-b9474802ce85.png" width="400px">
    
    In this image, the $y$ (blue) axis has rotated by $\frac{\pi}{2}$, causing the $x$ (red) and $z$ (green) axes to align. Thus rotations about both these axes have the same effect (object rotating about the length of its fuselage, notice the pivots for both these axes). No rotation about a single axis would now allow the object to *yaw*, or turn sideways, causing a loss of a degree of freedom.
    * **Consequences:**\
    Firstly, a frame in a gimbal lock loses a degree of freedom. Both the $x$ and $z$ axes now cause the object to roll about the length of its fuselage, and no axis allows it to yaw (turn towards the left or right). While it is always possible to still point it in any direction by using a combination of multiple rotations, the resulting motion would not be straight and smooth (since it has to be about multiple axes), which may be undesirable in applications such as animation.\
    Secondly, the alignment of the $z$ and $x$ axes means that a *yaw* (rotation about $z$) followed by a $\pm\frac{\pi}{2}$ *pitch* (rotation about $y$) produces the same result as a $\pm\frac{\pi}{2}$ *pitch* followed by a *roll* (rotation about $x$). Thus, Euler angles are not unique anymore, complicating their conversion to and from other representations of rotation.
<table>
    <tr>
        <td>
            <img src="https://user-images.githubusercontent.com/43912285/91632145-8922a180-e9fc-11ea-9064-8265224a740c.png" height="200px">
            <center>Pitch followed by roll.</center>
        </td>
        <td>
            <img src="https://user-images.githubusercontent.com/43912285/91632146-8aec6500-e9fc-11ea-8203-3f33e7e52a45.png" height="200px">
            <center>Yaw followed by pitch.</center>
        </td>
    </tr>
</table>

*Note: Each circle in these images rotates about its diameter, the one indicated by the pivot drawn in grey.*

- When you used `scipy.spatial.transform.Rotation` for the above 2 cases,
    - Have you used `zyx` above in `r.as_euler('')` argument? Why or why not? Explain the difference between extrinsic and instrinsic rotations with equivalent technical names from Craig book?
        * Ans: No, we used `'ZYX'`, and not `'zyx'` as the sequence of rotations. The former refers to *intrinsic* rotations, or rotations about moving axes (each rotation in a sequence of 3 rotations is about an axes which was affected by the previous rotation), which are defined as *Euler angles* in the book. `'zyx'`, on the other hand, refers to *extrinsic* rotations, or rotations about fixed axes (all the 3 rotations are about the original set of fixed axes), which are defined as *Fixed angles* in the book. 
    - Has `scipy` shown any warnings on any of the above cases? If yes, explain it.
        * Ans: While finding the Euler angles for $N$, `scipy` shows a warning indicating that it has detected a gimbal lock. Because Euler angles are no longer unique, it sets $\gamma=0$ (*roll*, or rotation about $x$) and then finds a set of Euler angles to achieve that rotation.
    - (Optional) For Case 3 above (quaternion) which you did using scipy, did you observe any problem with quaternion? Depending on your observations, which is better? Quaternion or Euler angles? And why?
         * Ans: Quarternions have a clear 2 to 1 mapping with rotation matrices ($q$ and $-q$ represent the same rotations in all cases), and do not suffer from degeneracies such as the gimbal lock. Thus, `scipy` was able to convert $N$ to quarternions and back, just as easily as $M$. Hence, quarternions are better than Euler angles for representing rotations for computations. However, quarternions come from a not-so-intuitive number system and are extremely hard to visualize. This is where Euler angles outperform quarternions.


## 2.3 Rotation matrix as an Operator
This question will help you in your understanding of [Rotator-transform (Vector-frame) equivalence](https://www.notion.so/saishubodh/Lecture-2-Transformations-11d69d8cef2d4cd195a98fa7d33224e1#f90ece4f5e374743bfed47e46a83ecfe).

![image.png](./misc/xyz-frame.png)
Consider the frame $XYZ$ in the above image. Say you have a vector $x_1=[0,\sqrt{3},0]$. Now you want to rotate it such that you end up at $x_2=[1,1,1]$ through a sequence of Euler angle rotations. Your goal is to find out those $\alpha, \beta \: \& \: \gamma$ ($ZYX$). We will follow this order whenever we refer to it below.

First, properly understand the so-called "Rotator-transform equivalence" to figure out what are the terms of rotation matrix. Then, put the math on paper and you will end up with a set of non-linear equations. Write the set of linear equations in LaTeX here:

$$ \textbf{Ans: Your set of equations here} $$
$$ \sqrt{3}\sin{\gamma} \cos{\beta} - 1 = 0 $$
$$ \sqrt{3}\cos{\alpha}\cos{\gamma} + \sqrt{3}\sin{\alpha}\sin{\beta}\sin{\gamma} - 1 = 0 $$
$$ -\sqrt{3}\sin{\alpha}\cos{\gamma} + \sqrt{3}\cos{\alpha}\sin{\beta}\sin{\gamma} - 1 = 0 $$

Solve these equations using `fsolve` from `scipy.optimize` as follows: (Come back and answer the following questions after coding it in the next block)
- `case1`: First, solve it with an initialization of (0,0,0). Check if your answer is correct using `np.isclose`.
    * What Euler angles did you get? Answer in $\alpha, \beta \: \& \: \gamma$ format:
        * Ans: *$a_1, b_1, c_1$ = [ -0.32906483356361393, -93.69048524053285, 0.74815097858118 ]* 
- `case2`: Now, forget about the solver for a moment: Can you visualize and think of sequence of rotations one by one to reach the final position (which is different than previous set of rotations)? Now, validate your answer by giving (your answer $\pm 5$) as initialization.
    * What Euler angles did you get? Answer in $\alpha, \beta \: \& \: \gamma$ format:
        * Ans: *$a_2, b_2, c_2$ = [5.49778714, 0, 0.61547971]*

In [17]:
##############################################################################
# DON'T EDIT
x_1 = np.array([0,np.sqrt(3),0])
x_2 = np.array([1,1,1])

In [18]:
##############################################################################
# TODO: Do tasks described in 2.3                                            #
#########a#####################################################################
# Replace "pass" statement with your code
def func(x):
    return [np.sqrt(3) * np.sin(x[2]) * np.cos(x[1]) - 1,
            np.sqrt(3) * np.cos(x[0]) * np.cos(x[2]) + np.sqrt(3) * np.sin(x[0]) * np.sin(x[1]) * np.sin(x[2]) - 1,
            -1 * np.sqrt(3) * np.sin(x[0]) * np.cos(x[2]) + np.sqrt(3) * np.cos(x[0]) * np.sin(x[1]) * np.sin(x[2]) -1
           ]

root1 = fsolve(func, [0 ,0 ,0])
a1, b1, c1 = root1
print(np.isclose(func(root1), [0, 0, 0]))

root2 = fsolve(func, [7*np.pi/4, 0, np.arcsin(1/np.sqrt(3))])
a2, b2, c2 = root2
print(np.isclose(func(root2), [0, 0, 0]))
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

[ True  True  True]
[ True  True  True]


In [19]:
root2

array([5.49778714, 0.        , 0.61547971])

In [20]:
# From Section 2.2, use the function which takes Euler angles and gives Rotation matrix as output.
# Uncomment and replace `r_mat` with the name of the function. (Do NOT edit anything else)
x_2_obtained_case1 = rotation_from_euler_zyx(a1,b1,c1) @ x_1.T #TODO: replace r_mat
x_2_obtained_case2 = rotation_from_euler_zyx(a2,b2,c2) @ x_1.T #TODO: replace r_mat
test = True #TODO: Set this as True

In [21]:
# DON'T EDIT
if test == True:
    case1_test = np.isclose(x_2_obtained_case1,  np.array([1.0,1.0,1.0]))
    case2_test = np.isclose(x_2_obtained_case2,  np.array([1.0,1.0,1.0]))
    print("For case 1, it is " + str(bool(case1_test[0] and case1_test[1] and case1_test[2])) + 
          " that I could end up at (1,1,1) after rotation.")
    print("For case 2, it is " + str(bool(case2_test[0] and case2_test[1] and case2_test[2])) + 
          " that I could end up at (1,1,1) after rotation.")

For case 1, it is True that I could end up at (1,1,1) after rotation.
For case 2, it is True that I could end up at (1,1,1) after rotation.


## 2.4 Gimbal Lock visualization (Optional)

A nice visualization video for gimbal lock is [this](https://www.youtube.com/watch?v=zc8b2Jo7mno). You are about to animate a similar visualization demonstrating gimbal lock 😃.

![image.png](./misc/xyz-frame.png)

- Write a function `two_four` for the visualization of gimbal lock. Follow the below steps to get the intuition going. Use Open3D for the following.
    - Say our frame's initial position is as the above image. Now the final goal at the end of rotation is to get the $Y$ axis pointing in the direction of the vector $(x,y,z)$ that you currently see in the above image. This point is fixed in space and is NOT moving as we rotate our axis.
    - For creating that point, you could use a small sphere using `open3d.geometry.create_mesh_sphere`. You already know how to create an axis by now.
    - Following our $ZYX$ convention, first rotate your frame about $Z$ axis by an angle, say $-35^{\circ}$. Then rotate about $Y$ axis by an angle ${\beta}$ and then about $X$ by say $55^{\circ}$.
        - Are there any specific angle(s) $\beta$ using which you will **never** reach our point $(x,y,z)$ ?
            - Clue: We are specifically talking about gimbal lock here and notice the word "never".
        - Under this (these) specific angle(s) of $\beta$ & different combinations of $\alpha$ and $\gamma$, make an animation and clearly show why your $Y$ axis is unable to align in the direction of that vector $(x,y,z)$ using the animation.

            If you are unsure to simulate the animation, you could do it as follows:

            - You could first fix some $\alpha$, say $-35^{\circ}$ & an above value of $\beta$, you can now vary $\gamma$ from $-180^{\circ} \text{ to }180^{\circ}$ to simulate the animation.
            - Now fix another $\alpha$, say $45^{\circ}$ and repeat the above process. So that's 2 specific values of $\alpha$.
            - Show this for all angles of $\beta$ if there are more than 1.
            - Therefore, when the code is run, there should be a minimum of (2 $\times$ (number of values of $\beta$)) animations. 2 for values of $\alpha$, you could show it for even more if you wish to.

In [22]:
##############################################################################
# TODO: Do tasks described in 2.4                                            #
##############################################################################

STEP_SIZE = np.radians(5) # degrees

def get_euler(obj, current, desired):
    current = np.array(current)
    desired = np.array(desired)
    
    ROA = obj.get_rotation_matrix_from_zyx(current)
    ROB = obj.get_rotation_matrix_from_zyx(current + desired)
    RAB = np.matmul(ROB, ROA.transpose())
    return RAB

def animate(obj, vis, center, current, desired):
    current = np.radians(np.array(current))
    desired = np.radians(np.array(desired))
    
    step = STEP_SIZE
    if desired[0] < 0:
        step = -step
    r = get_euler(obj, current, (step, 0, 0))
    for _ in range(int(abs(desired[0])/STEP_SIZE)): # ensure divisible
        obj.rotate(r, center=center)
        vis.update_geometry(obj)
        vis.poll_events()
        vis.update_renderer()
        time.sleep(0.15)
    
    current[0] += desired[0]
    
    step = STEP_SIZE
    if desired[1] < 0:
        step = -step
    r = get_euler(obj, current, (0, step, 0))
    for _ in range(int(abs(desired[1])/STEP_SIZE)): # ensure divisible
        obj.rotate(r, center=center)
        vis.update_geometry(obj)
        vis.poll_events()
        vis.update_renderer()
        time.sleep(0.15)
        
    current[1] += desired[1]
    
    step = STEP_SIZE
    if desired[2] < 0:
        step = -step
    r = get_euler(obj, current, (0, 0, step))
    for _ in range(int(abs(desired[2])/STEP_SIZE)): # ensure divisible
        obj.rotate(r, center=center)
        vis.update_geometry(obj)
        vis.poll_events()
        vis.update_renderer()
        time.sleep(0.15)

In [23]:
FRAME_SIZE = 1
TARGET = (FRAME_SIZE/1.3, FRAME_SIZE/1.3, FRAME_SIZE/1.3)
TARGET_SIZE = 0.03

def visualise_euler(euler_angles):
    world_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=FRAME_SIZE)
    our_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=FRAME_SIZE)
    target = o3d.geometry.TriangleMesh.create_sphere(radius=TARGET_SIZE)
    target.paint_uniform_color([0, 1, 1])
    target.translate(TARGET)

    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(world_frame)
    vis.add_geometry(our_frame)
    vis.add_geometry(target)
    vis.poll_events()
    vis.update_renderer()

    time.sleep(2)
    animate(our_frame, vis, (0, 0, 0), (0, 0, 0), euler_angles)

    time.sleep(2)
    vis.destroy_window()

def two_four():
    visualise_euler((-35, 90, 360))
    visualise_euler((-35, -90, 360))
    visualise_euler((45, 90, 360))
    visualise_euler((45, -90, 360))
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

> VOILA! You have just animated the famous Gimbal lock problem. If you are curious, read about the [Apollo 11](https://en.wikipedia.org/wiki/Gimbal_lock#On_Apollo_11) Gimbal lock incident.

### Questions for 2.4 (Optional)

- Mention the value(s) of $\beta$ here: 
    * Ans: Values of $\beta$ that cause a gimbal lock are: $\beta=\{\frac{\pi}{2}, -\frac{\pi}{2}\}$
- Now that you understand gimbal lock through visualization, explain it now in matrix form: For the above values of $\beta$, what does the rotation matrix look like? Can you explain why gimbal lock occurs from the rotation matrix alone? Clue: Use sin/cos formulae. 
    * Ans: The $Z-Y-X$ Euler angles to rotation matrix conversion looks like:
    
    $$
    R_{Z'Y'X'}(\alpha, \beta, \gamma) = \left[\begin{array}{ccc}
    c\alpha c\beta & c\alpha s\beta s\gamma - s\alpha c\gamma & c\alpha s\beta c\gamma + s\alpha s\gamma \\
    s\alpha c\beta & s\alpha s\beta s\gamma + c\alpha c\gamma & s\alpha s\beta c\gamma - c\alpha s\gamma \\
           -s\beta & c\beta s\gamma                           & c\beta c\gamma
    \end{array}\right]
    $$
    
    In which, on setting $\beta=\{\frac{\pi}{2}, -\frac{\pi}{2}\}$, we obtain:
    
    $$
    R_{Z'Y'X'}(\alpha, \pm\frac{\pi}{2}, \gamma) = \left[\begin{array}{ccc}
    0    & \pm c\alpha s\gamma - s\alpha c\gamma & \pm c\alpha c\gamma + s\alpha s\gamma \\
    0    & \pm s\alpha s\gamma + c\alpha c\gamma & \pm s\alpha c\gamma - c\alpha s\gamma \\
    \mp1 & 0                                     & 0
    \end{array}\right]
    $$
    
    Now, this is usually converted to Euler angles using the equations:
    
    $$
    \beta = tan^{-1} \left(-r_{31}, \sqrt{r_{11}^{2} + r_{21}^{2}}\right) \\
    \alpha = tan^{-1}(r_{21}/c\beta, r_{11}/c\beta) \\
    \gamma = tan^{-1}(r_{32}/c\beta, r_{33}/c\beta)
    $$
    
    In our case, with $r_{11}, r_{21}=0$, we get $\beta=\frac{\pi}{2}$, however the equations for $\alpha$ and $\gamma$ are degenerate and hence, not of much help. 
    
    Now, for $\beta=\frac{\pi}{2}$:
    
    $$
    r_{12} = c\alpha s\gamma - s\alpha c\gamma = sin(\gamma - \alpha) \\
    r_{22} = s\alpha s\gamma + c\alpha c\gamma = cos(\gamma - \alpha) \\
    \implies \gamma - \alpha = tan^{-1}(r_{12}, r_{22})
    $$
    
    So, a solution is:
    
    $$
    \beta = \frac{\pi}{2} \\
    \alpha = 0 \\
    \gamma = tan^{-1}(r_{12}, r_{22})
    $$
    
    Similarly, for $\beta = -\frac{\pi}{2}$:
    
    $$
    r_{12} = - c\alpha s\gamma - s\alpha c\gamma =  - sin(\gamma + \alpha) \\
    r_{22} = - s\alpha s\gamma + c\alpha c\gamma = cos(\gamma + \alpha) \\
    \implies \gamma + \alpha = - tan^{-1}(r_{12}, r_{22})
    $$
    
    So, a solution is:
    
    $$
    \beta = \frac{\pi}{2} \\
    \alpha = 0 \\
    \gamma = - tan^{-1}(r_{12}, r_{22})
    $$
    
    Thus, in the case when $\beta = \pm \frac{\pi}{2}$, only the sum of $\alpha$ and $\gamma$ can be determined, and not their unique values. This leads to a gimbal lock, where rotations about $z$ and $x$ have the same effect.
- Call the function `two_four` for the visualization of gimbal lock written above. 

In [ ]:
#uncomment the following and add input parameters if any
two_four()